# MERRA2 Daily Means Granularity Test
- Download
- Trim
- Create Cutout
- Generate Temperature Output

## 1. Imports and Create Dataset

In [ ]:
import calendar
import geodata
import xarray as xr
import logging
import pandas as pd
logging.basicConfig(level=logging.INFO)
DS_daily = geodata.Dataset(module="merra2",
					 years=slice(1998, 1998),
					 months=slice(1,1),
                     weather_data_config = "surface_flux_dailymeans")  


In [ ]:
if DS_daily.prepared == False:
	DS_daily.get_data()

## 2. Trim

In [ ]:
## Variables before trimming
ds = xr.open_dataset(DS_daily.downloadedFiles[0][1])
list(ds.data_vars)

In [ ]:
# Trim Variables
DS_daily.trim_variables()

In [ ]:
## Variables after trimming
ds = xr.open_dataset(DS_daily.downloadedFiles[0][1])
list(ds.data_vars)

In [ ]:
DS_daily.__dict__

In [ ]:
DS_daily.weatherconfig['variables'] 

## 3. Cutout

In [ ]:
cutout = geodata.Cutout(name="tokyo-2008-daily-test-7",
                       module="merra2",
                       weather_data_config="surface_flux_dailymeans",
                       xs=slice(138.5, 139.5),
                       ys=slice(35, 36),
                       years=slice(1998, 1998),
                       months=slice(1,1))

In [ ]:
cutout.prepare()

In [ ]:
cutout.__dict__

## 4.  Generate Outputs

In [ ]:
## Only Temperature output possible for Merra2 Daily Means currently

ds_temperature = geodata.convert.temperature(cutout)
ds_temperature

In [ ]:
df_temperature = ds_temperature.to_dataframe(name='temperature')
df_temperature

In [ ]:
df_temperature.to_csv('temperature_test_dailymeans.csv')